In [ ]:
import pandas as pd
import numpy as np

# Datos de muestra: Taquilla de Anime (Cifras simuladas para el ejercicio)
data = {
    'titulo': [
        'Demon Slayer: Mugen Train', 'Spirited Away', 'Your Name', 
        'Jujutsu Kaisen 0', 'Suzume', 'One Piece Film: Red', 
        'The First Slam Dunk', 'Howl\'s Moving Castle', 'Ponyo', 'Weathering With You'
    ],
    'estudio': [
        'Ufotable', 'Ghibli', 'CoMix Wave', 
        'MAPPA', 'CoMix Wave', 'Toei Animation', 
        'Toei Animation', 'Ghibli', 'Ghibli', 'CoMix Wave'
    ],
    'fecha_estreno': pd.to_datetime([
        '2020-10-16', '2001-07-20', '2016-08-26', 
        '2021-12-24', '2022-11-11', '2022-08-06', 
        '2022-12-03', '2004-11-20', '2008-07-19', '2019-07-19'
    ]),
    'recaudacion_jp_yen': [40430000000, 31680000000, 25030000000, 13800000000, 14790000000, 19700000000, 15730000000, 19600000000, 15500000000, 14190000000],
    'mercado_secundario': ['South Korea', 'USA', 'China', 'South Korea', 'China', 'France', 'South Korea', 'USA', 'Italy', 'China'],
    'calificacion_critica': [8.2, 9.5, 9.1, 7.8, 8.5, 7.5, 8.8, 8.9, 8.1, 8.0]
}

df = pd.DataFrame(data)

# Añadimos una columna calculada para complejidad: Recaudación en Millones (aprox)
df['recaudacion_millones'] = df['recaudacion_jp_yen'] / 1000000

display(df.head())

# Objetivo: Encontrar películas de 'CoMix Wave' o 'Ghibli' con calificación superior a 8.5
# y que NO hayan sido estrenadas antes del 2005.

filtro_query = df.query(
    "(estudio in ['CoMix Wave', 'Ghibli']) and (calificacion_critica > 8.5) and (fecha_estreno.dt.year >= 2005)"
)

print("--- Películas aclamadas de estudios específicos post-2005 ---")
display(filtro_query)

# Objetivo: Filtrar películas cuya recaudación está por encima del promedio general.
# Esto se calcula dinámicamente en el momento del filtrado.

filtro_dinamico = df.loc[lambda x: x['recaudacion_jp_yen'] > x['recaudacion_jp_yen'].mean()]

print(f"--- Películas con recaudación superior al promedio ({df['recaudacion_jp_yen'].mean()/1e9:.2f} mil millones) ---")
display(filtro_dinamico[['titulo', 'recaudacion_millones', 'estudio']])

# Objetivo: Buscar títulos que contengan palabras relacionadas con "Mundo" o "Elementos" (ej. Weathering, Moving, World)
# Utilizando expresiones regulares (Regex). Ignorando mayúsculas/minúsculas.

# Patrón regex: busca palabras que terminen en 'ing' O contengan 'Slam'
patron = r'(ing\b)|(Slam)'

filtro_regex = df[df['titulo'].str.contains(patron, regex=True, case=False)]

print("--- Títulos que coinciden con el patrón Regex ---")
display(filtro_regex[['titulo', 'fecha_estreno']])

# Objetivo: Seleccionar la película con MAYOR recaudación de CADA estudio.
# 1. Agrupamos por estudio.
# 2. Transformamos para obtener el máximo de cada grupo en todas las filas.
# 3. Filtramos donde la recaudación individual iguala al máximo de su grupo.

filtro_grupo = df[df['recaudacion_jp_yen'] == df.groupby('estudio')['recaudacion_jp_yen'].transform('max')]

print("--- La película más taquillera de cada estudio ---")
display(filtro_grupo[['estudio', 'titulo', 'recaudacion_millones']].sort_values(by='recaudacion_millones', ascending=False))

# Objetivo: Excluir películas estrenadas durante la pandemia (2020-2021)
# PERO mantenerlas si tuvieron un éxito masivo en Corea del Sur (Mercado Secundario).

inicio_pandemia = '2020-01-01'
fin_pandemia = '2021-12-31'

# Máscara 1: Estrenada en pandemia
en_pandemia = (df['fecha_estreno'] >= inicio_pandemia) & (df['fecha_estreno'] <= fin_pandemia)

# Máscara 2: Éxito en Corea del Sur
es_korea = df['mercado_secundario'] == 'South Korea'

# Lógica: (NO en pandemia) O (En pandemia Y es Korea)
filtro_complejo = df[~en_pandemia | (en_pandemia & es_korea)]

print("--- Películas fuera de pandemia (o excepciones de Corea del Sur) ---")
display(filtro_complejo[['titulo', 'fecha_estreno', 'mercado_secundario']])